In [1]:
import torch as th
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


In [2]:
workers

[<VirtualWorker id:bob #objects:14>,
 <VirtualWorker id:theo #objects:14>,
 <VirtualWorker id:jason #objects:14>,
 <VirtualWorker id:alice #objects:14>,
 <VirtualWorker id:andy #objects:14>,
 <VirtualWorker id:jon #objects:14>]

In [3]:
boston_data = grid.search("#boston", "#data")
boston_target = grid.search("#boston", "#target")

In [4]:
n_features = boston_data['alice'][0].shape[1]
n_targets = 1

model = th.nn.Linear(n_features, n_targets)

In [5]:
# Cast the result in BaseDatasets
datasets = []
for worker in boston_data.keys():
    dataset = sy.BaseDataset(boston_data[worker][0], boston_target[worker][0])
    datasets.append(dataset)

# Build the FederatedDataset object
dataset = sy.FederatedDataset(datasets)
print(dataset.workers)
optimizers = {}
for worker in dataset.workers:
    optimizers[worker] = th.optim.Adam(params=model.parameters(),lr=1e-2)

['bob', 'theo', 'jason', 'alice', 'andy', 'jon']


In [6]:
train_loader = sy.FederatedDataLoader(dataset, batch_size=32, shuffle=False, drop_last=False)

In [8]:
epochs = 10
for epoch in range(1, epochs + 1):
    loss_accum = 0
    for batch_idx, (data, target) in enumerate(train_loader):       # 使用数据加载器
        model.send(data.location)
        
        optimizer = optimizers[data.location.id]
        optimizer.zero_grad()
        pred = model(data)
        loss = ((pred.view(-1) - target)**2).mean()
        loss.backward()
        optimizer.step()
        
        model.get()
        loss = loss.get()
        
        loss_accum += float(loss)
        
        if batch_idx % 8 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tBatch loss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))            
            
    print('Total loss', loss_accum)

Train Epoch: 1 [0/16 (0%)]	Batch loss: 2367.532715
Train Epoch: 1 [8/16 (50%)]	Batch loss: 86.179184
Train Epoch: 1 [16/16 (100%)]	Batch loss: 189.112701
Total loss 6982.528579711914
Train Epoch: 2 [0/16 (0%)]	Batch loss: 132.243729
Train Epoch: 2 [8/16 (50%)]	Batch loss: 82.996765
Train Epoch: 2 [16/16 (100%)]	Batch loss: 122.085678
Total loss 3281.2114601135254
Train Epoch: 3 [0/16 (0%)]	Batch loss: 54.023983
Train Epoch: 3 [8/16 (50%)]	Batch loss: 96.907722
Train Epoch: 3 [16/16 (100%)]	Batch loss: 77.894753
Total loss 1803.068790435791
Train Epoch: 4 [0/16 (0%)]	Batch loss: 61.169136
Train Epoch: 4 [8/16 (50%)]	Batch loss: 103.411514
Train Epoch: 4 [16/16 (100%)]	Batch loss: 27.831673
Total loss 1224.775598526001
Train Epoch: 5 [0/16 (0%)]	Batch loss: 55.768658
Train Epoch: 5 [8/16 (50%)]	Batch loss: 67.539917
Train Epoch: 5 [16/16 (100%)]	Batch loss: 27.874121
Total loss 1138.0568761825562
Train Epoch: 6 [0/16 (0%)]	Batch loss: 51.960052
Train Epoch: 6 [8/16 (50%)]	Batch loss: 60.